In [80]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

In [8]:
evaluator = tf.keras.models.load_model("drive/MyDrive/MIT687/MICPredictor.h5",
                                       custom_objects={"UsefulFraction":UsefulFraction()})
aa_vocal = np.load("drive/MyDrive/MIT687/SeqTV_vocal.npy")
labelencoder = tf.keras.layers.StringLookup(vocabulary=aa_vocal[1:], invert=True, oov_token='')

In [65]:
class UsefulFraction(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, name='UsefulFraction', **kwargs):
        super(UsefulFraction, self).__init__(name=name, **kwargs)
        self.threshold = threshold
        self.total_count = self.add_weight(name='total_count', initializer='zeros')
        self.within_threshold_count = self.add_weight(name='within_threshold_count',
                                                      initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        error = tf.abs(y_true - y_pred)
        within_threshold = tf.cast(tf.less(error, self.threshold), tf.float32)
        self.total_count.assign_add(tf.reduce_sum(tf.ones_like(error)))
        self.within_threshold_count.assign_add(tf.reduce_sum(within_threshold))

    def result(self):
        return self.within_threshold_count / self.total_count

    def reset_states(self):
        self.total_count.assign(0)
        self.within_threshold_count.assign(0)

class Evaluator(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
        self.loss=evaluator

    def call(self, y_true, y_pred):
        loss0 = self.loss([y_pred, y_true])
        return loss0

In [154]:
def create_model():
    inputs = tf.keras.layers.Input((326,),name="StateInput")
    x = tf.keras.layers.Dense(512, activation="relu", name="Dense_0")(inputs)
    x = tf.keras.layers.Dense(190, activation="relu", name="Dense_1")(x)
    x = tf.keras.layers.Reshape((190, 1), name="Reshape")(x)
    x = tf.keras.layers.LSTM(256, return_sequences=True, name="GRU_0")(x)
    x = tf.keras.layers.LSTM(256, return_sequences=True, name="GRU_1")(x)
    x = tf.keras.layers.Dense(256, activation="relu", name="Dense_2")(x)
    x = tf.keras.layers.Dense(43, activation="softmax", name="Output")(x)
    model = tf.keras.models.Model(inputs, x, name="Generator")
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=Evaluator())
    return model

In [176]:
model = create_model()
for epoch in range(10):
  np.random.seed(8701)
  zeros = np.zeros(326)
  states = []
  for i in range(10240):
      state = np.zeros(326)
      state[np.random.randint(0, 326)]=1
      states.append([state])
  states = np.concatenate(states, axis=0)
  model.fit(states, states, epochs=1, batch_size=32)

320/320 [==============================] - 8s 24ms/step - loss: 4.2149


In [177]:
pred = model.predict(states[0:1])
labelencoder(tf.math.argmax(pred, axis=2)).numpy().astype('str')

1/1 [==============================] - 1s 1s/step


array([['s', 's', 's', 's', 's', 's', 's', 's', 's', 's', 'L', 's', 's',
        's', 's', 's', 's', 's', 's', 's', 's', 's', 's', 'L', 's', 'k',
        'L', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 's', 's', 's',
        's', 's', 'L', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'L',
        'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'L',
        'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'L', 'k', 'k', 'k', 'k',
        'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 's', 's', 's', 's', 's',
        's', 'L', 's', 's', 's', 'L', 'L', 'L', 'k', 'k', 'k', 'k', 'k',
        'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'L', 'k', 'k', 'k', 'k',
        'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 's', 's', 's', 's', 's',
        's', 'L', 'L', 'L', 's', 's', 's', 's', 's', 's', 'L', 'L', 'L',
        'L', 'L', 'L', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k',
        'k', 'k', 's', 's', 'L', 'L', 'k', 's', 's', 's', 'L', 'k', 's',
        's', 's', 's', 's', 's', 's', 's', 's', 'L'